In [2]:
import numpy as np
from math import pi, tau

from datetime import datetime, timezone
from tzwhere import tzwhere
import pytz
import suncalc

from colorsys import hls_to_rgb
from PIL import Image

In [27]:
# azimuth, altitude to color
def get_color(azimuth, altitude, sunrise_jump=0.2, hue_shift=0.0, as_hex=True):
    # ranges from 0 (no jump) to 1 (day is all white, night all black)
    assert 0 <= sunrise_jump <= 1, "sunrise_jump must be between 0 and 1 inclusive"

    # can be any float, but values outside the range [0, 1) are redundant
    # shifts all hues in the r -> g -> b -> r direction
    assert isinstance(hue_shift, float), "hue_shift must be a float"
    
    print(f'azi: {azimuth}')
    print(f'alt: {altitude}')

    # yes, this could be simplified, and no, don't try to do it please.
    altitude_scaled = (altitude / pi) * 2  # range [-1, 1]
    print(altitude_scaled)
    
    altitude_scaled *= 1 - sunrise_jump
    print(altitude_scaled)
    
    altitude_scaled += sunrise_jump * (1 if altitude >= 0 else -1)  # range [-1, 1]
    print(altitude_scaled)
    
    lightness = altitude_scaled / 2 + 0.5  # range [0, 1]
    print(lightness)

    hue = ((azimuth / tau) + 0.5 + hue_shift) % 1  # range [0, 1]
    print(hue)

    r, g, b = hls_to_rgb(hue, lightness, 1)
    r = round(255 * r)
    g = round(255 * g)
    b = round(255 * b)
    return "{0:02x}{1:02x}{2:02x}".format(r, g, b).upper() if as_hex else r, g, b

In [43]:
# azimuth, altitude to color
def get_color_vec(azimuth, altitude, sunrise_jump=0.2, hue_shift=0.0):
    # ranges from 0 (no jump) to 1 (day is all white, night all black)
    assert 0 <= sunrise_jump <= 1, "sunrise_jump must be between 0 and 1 inclusive"

    # can be any float, but values outside the range [0, 1) are redundant
    # shifts all hues in the r -> g -> b -> r direction
    assert isinstance(hue_shift, float), "hue_shift must be a float"
    
    print(f'azi: {azimuth}')
    print(f'alt: {altitude}')

    # yes, this could be simplified, and no, don't try to do it please.
    altitude_scaled = (altitude / pi) * 2  # range [-1, 1]
    print(altitude_scaled)
    
    altitude_scaled *= 1 - sunrise_jump
    # altitude_scaled = altitude_scaled * (1 - sunrise_jump)
    print(altitude_scaled)
    
    # altitude_scaled += sunrise_jump * (1 if altitude >= 0 else -1)  # range [-1, 1]
    idx_alt_pos = altitude_scaled >= 0
    idx_alt_neg = altitude_scaled < 0
    altitude_scaled[idx_alt_pos] += sunrise_jump * 1
    altitude_scaled[idx_alt_neg] += sunrise_jump * -1
    print(altitude_scaled)
    
    lightness = altitude_scaled / 2 + 0.5  # range [0, 1]
    print(lightness)

    hue = ((azimuth / tau) + 0.5 + hue_shift) % 1  # range [0, 1]
    print(hue)
    
    saturation = np.ones(hue.shape)
    
    r, g, b = np.vectorize(hls_to_rgb)(hue, lightness, saturation)
    
    r = np.round(255 * r)
    g = np.round(255 * g)
    b = np.round(255 * b)
    
    return r, g, b

In [29]:
azi = 0.37464615
alt = -0.4540797

In [45]:
get_color(azi, alt, sunrise_jump=0.2, hue_shift=0.0, as_hex=False)

azi: 0.37464615
alt: -0.4540797
-0.2890761152507396
-0.2312608922005917
-0.4312608922005917
0.28436955389970414
0.5596267866828477


(0, 93, 145)

In [44]:
azi2 = np.array([azi])
alt2 = np.array([alt])
get_color_vec(azi2, alt2, sunrise_jump=0.2, hue_shift=0.0)

azi: [0.37464615]
alt: [-0.4540797]
[-0.28907612]
[-0.23126089]
[-0.43126089]
[0.28436955]
[0.55962679]


(array([0.]), array([93.]), array([145.]))